<a href="https://colab.research.google.com/github/Fuenfgeld/DMA2023TeamD/blob/dev_branch_EDA_Prediction/3_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Objective**: Exploratory Data Analysis (EDA)

Exploratory data analysis is a task performed by data scientists to get familiar with the data. All the initial tasks you do to understand your data well are known as EDA.

There are main components of exploring data:
1. Understanding data/Basic Data Exploration
2. Cleaning dataset
3. Understanding variables
4. Analyzing relationships between variables



# Importing Libraries

In [ ]:
# install the newest version,  
# uncomment this line in first runtime, it needs to restart runtime
!pip3 install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip


In [ ]:
import ydata_profiling
ydata_profiling.version.__version__
# => 2.9.0 aber nun 0.0.dev0

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
# from datetime import datetime
# import re
import seaborn as sns
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
# will make plot outputs appear and stored within the notebook.
%matplotlib inline

In [ ]:
from google.colab import drive
# mount drive to access database
drive.mount("/content/drive")

In [ ]:
material_path = "/content/drive/Shareddrives/Projektarbeit DMA Gruppe D"

In [ ]:
# list of datawarehoses
!ls "/content/drive/Shareddrives/Projektarbeit DMA Gruppe D/DWH_dbs"


# Loading Data


In [ ]:
# datawarehouse path
DB_DWH_PATH = f"{material_path}/DWH_dbs/DWH_allergy_test.db"
# DB_DWH_PATH = "/content/drive/My Drive/synthea_patient_data/DWH_dbs/DWH_lung_cancer.db"

# connect to db
dwh_conn = sqlite3.connect(DB_DWH_PATH)

In [ ]:
# list of tables in db
if dwh_conn is not None:
  dwh_cursor = dwh_conn.cursor()
  dwh_cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
  print("List of Tables", dwh_cursor.fetchall())


In [ ]:
# Read sqlite query results into a pandas DataFrame
# demographic data
df_patients = pd.read_sql_query("SELECT * FROM PATIENTS_INFO", dwh_conn)
# diagnoses data
df_conditions = pd.read_sql_query("select * from conditions_info", dwh_conn)

In [ ]:
# MERGE TWO DATASETS 
df = pd.merge(df_patients, df_conditions, left_on="Id", right_on="PATIENT", how="inner")

In [ ]:
# close db connection
# dwh_conn.close()

# Understanding Data/Basic Data Exploration
In this step, we will perform the below operations to check what the data set comprises of. We will check the below things:

* head of the dataset
* the shape of the dataset
* info of the dataset
* summary of the dataset
* (n)unique values for each variable

In [ ]:
# .head() returns the first 5 rows of my dataset. This is useful if you want to see some example values for each variable.
df.head()

In [ ]:
# .shape returns the number of rows by the number of columns
#size of dataset
df.shape

In [ ]:
# .columns returns the name of all of your columns in the dataset.
df.columns

In [ ]:
# attribute type
df.info()

The described method will help to see how data has been spread for numerical values. We can clearly see the minimum value, mean values, different percentile values, and maximum values.

In [ ]:
# .describe summarizes the count, mean, standard deviation, min, and max for numerical variables.
df.describe()

In [ ]:
# .nunique(axis=0) returns the number of unique values for each variable.
df.nunique(axis=0)


In [ ]:
# replacing blank values (with space) with NAN
df = df.replace(r'^\s*$', np.nan, regex=True)


In [ ]:
# profile = ProfileReport(df)
# profile

#  Cleaning Dataset


### Removing Duplicate Rows


In [ ]:
print("Number of Duplicated Rows", df.duplicated(df.columns).sum())

In [ ]:

# dropping duplicate values if exists and keep first one
df.drop_duplicates(keep="first",inplace=True) 
print("Size of dataset after removinf duplicated rows", df.shape)
  

### Removing Redundant and Unnecessary Variables

In [ ]:
df.columns

In [ ]:
df = df.drop(['SSN', 'PREFIX', 'ZIP', 'DRIVERS', 'PASSPORT', 'FIRST',
              'LAST', 'BIRTHPLACE', 'ADDRESS', 'STATE', 'COUNTRY', 
              'PATIENT', 'ENCOUNTER'], axis=1)

### Handling Missing Value

We can see that we have various missing values in the respective columns. There are various ways of treating your missing values in the data set. And which technique to use when is actually dependent on the type of data you are dealing with.

* Drop the missing values: In this case, we drop the missing values from those variables. In case there are very few missing values you can drop those values.
* Impute with mean value: For the numerical column, you can replace the missing values with mean values. Before replacing with mean value, it is advisable to check that the variable shouldn’t have extreme values .i.e. outliers.
* Impute with median value: For the numerical column, you can also replace the missing values with median values. In case you have extreme values such as outliers it is advisable to use the median approach.
* Impute with mode value: For the categorical column, you can replace the missing values with mode values i.e the frequent ones.

In [ ]:
df.shape

In [ ]:
# percentage of not null values in each column
df.count()/len(df) * 100

In [ ]:
# number of null values in each column
df.isnull().sum()

In [ ]:
# delete sparc columns
df.drop(['SUFFIX', 'MAIDEN'], axis=1, inplace=True)

In [ ]:
# fill nall values with todays date
df["DEATHDATE"] = df.DEATHDATE.fillna(pd.to_datetime("today"))
df["STOP"] = df.STOP.fillna(pd.to_datetime("today"))

In [ ]:
# fill nall values with mode
df['MARITAL'].fillna(df['MARITAL'].mode()[0], inplace=True)

In [ ]:
# convert to date
df["DEATHDATE"] = pd.to_datetime(df["DEATHDATE"])
df["BIRTHDATE"] = pd.to_datetime(df["BIRTHDATE"])
df["START"] = pd.to_datetime(df["START"])
df["STOP"] = pd.to_datetime(df["STOP"])

In [ ]:
# calculate age 
df["AGE"] = df.DEATHDATE.dt.year - df.BIRTHDATE.dt.year
# df["DURATION_day"] = (df.STOP.dt.year - df.START.dt.year) * 12 + (df.STOP.dt.month - df.START.dt.month)
df["DURATION"] = df.STOP.dt.to_period("M").astype(int) - df.START.dt.to_period("M").astype(int)

df.DURATION

In [ ]:
# number of null values in each column
df.isnull().sum()

# Univariate Analysis

## Grouping

In [ ]:
# number of patients in procedure table
df.groupby(["Id"]).size()

There are info for 133 patients in table.



In [ ]:
df.groupby(["Id", "CODE"]).size()

For some patients, there are more than one diagnose.

## Histogram

if you only wanted to explore a single variable by itself? This is when histograms come into play.


In [ ]:
df['AGE'].plot(kind='hist', bins=20, figsize=(12,6), facecolor='grey',edgecolor='black')

In [ ]:
# # binning age column and remove age attribute
# bins = [i for i in range(df.AGE.min(), df.AGE.max(), 5)]
# df['AGE_BINS'] = pd.cut(x=df['AGE'], bins=bins)

# df.drop(["AGE"], axis=1, inplace=True)


## Boxplot and Removing Outlier

We can discover outliers with visualization tools:
* Box plot
* Scatter plot
* Z-Score
* IQR Score

For example:

In [ ]:
sns.boxplot(x=df["DURATION"])

In [ ]:
df = df[df["DURATION"]<1200]
df.shape

## Countplot

In [ ]:
# distribution of GENDER attribute
sns.countplot(x="variable", hue= "value", data=pd.melt(df[["GENDER"]]))

In [ ]:
# distribution of Marital attribute
sns.countplot(x="variable", hue= "value", data=pd.melt(df[["MARITAL"]]))

In [ ]:
# distribution of RACE attribute
sns.countplot(x="variable", hue= "value", data=pd.melt(df[["RACE"]]))

In [ ]:
# distribution of RACE attribute
sns.countplot(x="variable", hue= "value", data=pd.melt(df[["ETHNICITY"]]))

In [ ]:
# convert categorical variable to numerical by replacing
df.GENDER.replace(['M', 'F'], [0, 1], inplace=True)
df.MARITAL.replace(['M', 'S'], [0, 1], inplace=True)
df.RACE.replace(["white", "black", "asian"], [1, 2, 3], inplace=True)
df.ETHNICITY.replace(["nonhispanic", "hispanic"], [0, 1], inplace=True)

#  Multivariate Analysis: Analyzing Relationships Between Variables

Correlation matrices and scatterplots are useful for exploring the relationship between two variables.


### Correlation Matrix


In [ ]:
# list of columns
df.info()

In [ ]:
# Generating the correlating matrix
corr = df.corr()

plt.figure(figsize=(7, 5))
# Generating the correlation heat-map
sns.heatmap(corr, annot=False)

In [ ]:
# df subset contains numerical variables
numdf = df.select_dtypes(include=np.number)
# df subset contains categorical variables
catdf = df.select_dtypes(exclude=np.number)
numdf.shape, catdf.shape

In [ ]:
# select feature with correlation less than a threshold
columns = np.full((corr.shape[0],), True, dtype=bool)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= 0.7:
            if columns[j]:
                columns[j] = False

selected_columns = numdf.columns[columns]
selected_columns

* It is obvious correlation between GENDER and SCT_CODE_PROCEDURE < correlation between SCT_CODE_PROCEDURE and AGE.

*   AGE and GENDER are correlated.



### Scatterplot


In [ ]:
df.plot(kind='scatter', y='HEALTHCARE_EXPENSES', x='AGE')


In [ ]:
df = df[(df["AGE"]<100)]

In [ ]:
df.plot(kind='scatter', y='DURATION', x='AGE')


In [ ]:
df.plot(kind='scatter', y='HEALTHCARE_EXPENSES', x='AGE')


In [ ]:
df.plot(kind='scatter', y='AGE', x='RACE')


In [ ]:
df.plot(kind='scatter', y='HEALTHCARE_EXPENSES', x='RACE')


In [ ]:
df.plot(kind='scatter', x='MARITAL', y='AGE')


In [ ]:
df.plot(kind='scatter', x='HEALTHCARE_EXPENSES', y='GENDER')


In [ ]:
# sns.pairplot(df_selected)

### PCA
Principal components are the key to PCA; It is used on two use-case:
* Data visualization
* Speeding machine learning algorithms

PCA is effected by scale so you need to scale the features in your data before applying PCA. Use StandardScaler to help you standardize the dataset’s features onto unit scale (mean = 0 and variance = 1) 

In [ ]:
# select some columns
df_selected = df[[ "MARITAL", "RACE", "ETHNICITY", "GENDER", "AGE", 'DURATION', 'CODE', "HEALTHCARE_EXPENSES", 'HEALTHCARE_COVERAGE']]

In [ ]:
from sklearn.preprocessing import StandardScaler


features = ['MARITAL', 'RACE', 'ETHNICITY', 'GENDER', 'AGE', 'DURATION', 'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE']
# # Separating out the features
x = df_selected.loc[:, features].values
# # Separating out the target
y = df_selected.loc[:,['CODE']].values
# # Standardizing the features
x = StandardScaler().fit_transform(x)

In [ ]:
# check normalization
x.shape, np.mean(x), np.std(x)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

In [ ]:
finalDf = pd.concat([principalDf, df_selected[['CODE']]], axis = 1)
finalDf.head()

In [ ]:
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))


From the above output, you can observe that the principal component 1 holds 31.6% of the information while the principal component 2 holds only 18% of the information. Also, the other point to note is that while projecting  data to a two-dimensional data, 50% information was lost.

In [ ]:
df_selected.CODE

In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = ['MARITAL', 'RACE', 'ETHNICITY''GENDER', 'AGE']
targets = [19169002,162864005, 197927001, 10509002, 444814009]
colors = ['r', 'g', 'b', 'g', 'c']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf['CODE'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()

# Links
https://chrisalbon.com/python/basics/strings_to_datetime/

https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html

 EDA: 
 
 https://towardsdatascience.com/a-gentle-introduction-to-exploratory-data-analysis-f11d843b8184

 https://towardsdatascience.com/an-extensive-guide-to-exploratory-data-analysis-ddd99a03199e

 https://www.analyticssteps.com/blogs/how-do-exploratory-data-analysis-building-machine-learning-models


 https://www.analyticsvidhya.com/blog/2020/08/exploratory-data-analysiseda-from-scratch-in-python/


 PCA

 https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60